### DataPrep for Modeling with Cutoff Points

Notes:

* A000 Data Exploration to understand how the data looks like
* A001 Statistical Significance
* A002 Week 16 Data Prep for Modeling at identified CutOff Points 

For EEE, Let's try to opt for the following at Week 16 cutoff point for modeling point

* EEE 2013J 1052 students for train week
* EEE 2014B 694 students for train week
* EEE 2014J 1188 students for test week

##### Not Doing
For FFF, Let's try to opt for the following at Week 12 cutoff point for modeling point 
* FFF 2013B 1614 students for train week 
* FFF 2013J 2283 students for train week 
* FFF 2014B 1500 students for train week 
* FFF 2014J 2365 students for test week

### Import and Set Libraries

In [12]:
import numpy as np
import pandas as pd
import os
import math
print (pd.__version__)
print (np.__version__)
print (os.name)
print(hasattr(math, "isfinite"))
os.getcwd()
os.chdir('C:\\Users\\EeLian\\Desktop\\OULAD_KAGGLE\\PythonCodeCleanUp\\Data')

1.2.4
1.20.1
nt
True


### Data Preparation: Student Info dataset

In [13]:
# read datafile
studentInfo = pd.read_csv('studentInfo.csv')
courses = pd.read_csv('courses.csv')
studentRegistration = pd.read_csv('studentRegistration.csv')

# add _fl_latest_module_attended
studentInfo
studentInfo.sort_values(by= ['id_student','code_module','code_presentation'], inplace = True, ascending = False)
studentInfo['_fl_latest_module_attended'] = studentInfo.groupby(['id_student','code_module'])['id_student'].rank(method='first', ascending = False)
# for checking purposes
# studentInfo[studentInfo['id_student'].isin([584077])]

# merge/ left join to get studentInfo + courses + registration 
# name as studentInfo3 cos 3 files
studentInfo2 = studentInfo.merge(courses, how='left', left_on=["code_module", "code_presentation"], right_on=["code_module", "code_presentation"])
studentInfo3 = studentInfo2.merge(studentRegistration, how='left', left_on=["code_module", "code_presentation","id_student"], right_on=["code_module", "code_presentation","id_student"])

# identifying actual dropout. dropout after semester starts=actual dropout; else, it's deregistration
# studentInfo3['_flag_actual_dropout']=studentInfo3['date_unregistration'].isnull().astype(int)
studentInfo3.loc[studentInfo3['date_unregistration'] >0, '_flag_actual_dropout'] = 1
studentInfo3.loc[studentInfo3['date_unregistration'] <=0, '_flag_actual_dropout'] = 0
studentInfo3.loc[studentInfo3['date_unregistration'].isnull(), '_flag_actual_dropout'] = 0

# export to csv
# studentInfo3.to_csv('DatasetForModel_StudentInfo.csv',index=True,header=True)

##### What we want to prepare

##### Aggregated Dimension
* id_student
* code_module
* code_presentation

##### Aggregated Metrics
* count_total_assessment
* count_passed_assessment_CMA
* count_passed_assessment_TMA
* count_passed_assessment_Exam
* count_passed_assessment_Total
* count_failed_assessment_CMA
* count_failed_assessment_TMA
* count_failed_assessment_Exam
* count_failed_assessment_Total
* rate_passed_assessment
* sum_weighted_score
* sum_score

In [14]:
# import data
studentAssessment = pd.read_csv('studentAssessment.csv')
assessments = pd.read_csv('assessments.csv')

# merge/ left join
# name as studentAssessment2
studentAssessment2 = studentAssessment.merge(assessments, how='left', left_on=["id_assessment"], right_on=["id_assessment"])

# add _flag_null_score just to check if there is submission
studentAssessment2['_flag_null_score']=studentAssessment2['score'].isnull().astype(int)

# add weighted score
studentAssessment2['_weighted_score'] = studentAssessment2['score']*studentAssessment2['weight']

# add pass fail. the NaN means didnt submit
studentAssessment2.loc[studentAssessment2['score'] >=40, '_flag_assessment_passed'] = 1
studentAssessment2.loc[studentAssessment2['score'] <40, '_flag_assessment_passed'] = 0
studentAssessment2["_date_group"] = pd.Series([], dtype=float)
studentAssessment2["_date_group"] = studentAssessment2['date_submitted']
studentAssessment2.loc[studentAssessment2['date_submitted'] <0, '_date_group'] = studentAssessment2['_date_group'].apply(lambda x: math.ceil(float(x/7)))
studentAssessment2.loc[studentAssessment2['date_submitted'] >=0, '_date_group'] = studentAssessment2['_date_group'].apply(lambda x: math.floor(float((x/7)+1)))
#studentAssessment2.head()

# for EEE Course, Cut off filter at Week 16
studentAssessment2 = studentAssessment2[(studentAssessment2['_date_group']<=16)]
#studentAssessment2.to_csv('studentAssessment2_week16.csv',index=True,header=True)

In [15]:
# Feature Engineering
# transforming multi-rows input to individual vertical columns

tbl_total_weighted_score=studentAssessment2.\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'_weighted_score':['sum']})
tbl_total_weighted_score.columns = ['sum_weighted_score']
tbl_total_weighted_score = tbl_total_weighted_score.reset_index()

tbl_total_score=studentAssessment2.\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'score':['sum']})
tbl_total_score.columns = ['sum_score']
tbl_total_score = tbl_total_score.reset_index()

tbl_total_assessment=studentAssessment2.\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'_flag_assessment_passed':['count']})
tbl_total_assessment.columns = ['count_total_assessment']
tbl_total_assessment = tbl_total_assessment.reset_index()

##################################

tbl_passed_assessment_CMA=studentAssessment2\
            [(studentAssessment2['_flag_assessment_passed'].isin([1]))&\
             (studentAssessment2['assessment_type'].isin(['CMA']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'_flag_assessment_passed':['count']})
tbl_passed_assessment_CMA.columns = ['count_passed_assessment_CMA']
tbl_passed_assessment_CMA = tbl_passed_assessment_CMA.reset_index()

tbl_passed_assessment_TMA=studentAssessment2\
            [(studentAssessment2['_flag_assessment_passed'].isin([1]))&\
             (studentAssessment2['assessment_type'].isin(['TMA']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'_flag_assessment_passed':['count']})
tbl_passed_assessment_TMA.columns = ['count_passed_assessment_TMA']
tbl_passed_assessment_TMA = tbl_passed_assessment_TMA.reset_index()

tbl_passed_assessment_Exam=studentAssessment2\
            [(studentAssessment2['_flag_assessment_passed'].isin([1]))&\
             (studentAssessment2['assessment_type'].isin(['Exam']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'_flag_assessment_passed':['count']})
tbl_passed_assessment_Exam.columns = ['count_passed_assessment_Exam']
tbl_passed_assessment_Exam = tbl_passed_assessment_Exam.reset_index()

tbl_passed_assessment_Total=studentAssessment2\
            [(studentAssessment2['_flag_assessment_passed'].isin([1]))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'_flag_assessment_passed':['count']})
            #agg({'_flag_assessment_passed':['sum','count'],'score':['count']})
tbl_passed_assessment_Total.columns = ['count_passed_assessment_Total']
tbl_passed_assessment_Total = tbl_passed_assessment_Total.reset_index()

##################################

tbl_failed_assessment_CMA=studentAssessment2\
            [((studentAssessment2['_flag_assessment_passed'].isin([0]))|(studentAssessment2['_flag_assessment_passed'].isnull()))&\
             (studentAssessment2['assessment_type'].isin(['CMA']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'_flag_assessment_passed':['count']})
tbl_failed_assessment_CMA.columns = ['count_failed_assessment_CMA']
tbl_failed_assessment_CMA = tbl_failed_assessment_CMA.reset_index()

tbl_failed_assessment_TMA=studentAssessment2\
            [((studentAssessment2['_flag_assessment_passed'].isin([0]))|(studentAssessment2['_flag_assessment_passed'].isnull()))&\
             (studentAssessment2['assessment_type'].isin(['TMA']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'_flag_assessment_passed':['count']})
tbl_failed_assessment_TMA.columns = ['count_failed_assessment_TMA']
tbl_failed_assessment_TMA = tbl_failed_assessment_TMA.reset_index()

tbl_failed_assessment_Exam=studentAssessment2\
            [((studentAssessment2['_flag_assessment_passed'].isin([0]))|(studentAssessment2['_flag_assessment_passed'].isnull()))&\
             (studentAssessment2['assessment_type'].isin(['Exam']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'_flag_assessment_passed':['count']})
tbl_failed_assessment_Exam.columns = ['count_failed_assessment_Exam']
tbl_failed_assessment_Exam = tbl_failed_assessment_Exam.reset_index()

tbl_failed_assessment_Total=studentAssessment2\
            [((studentAssessment2['_flag_assessment_passed'].isin([0]))|(studentAssessment2['_flag_assessment_passed'].isnull()))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'_flag_assessment_passed':['count']})
            #agg({'_flag_assessment_passed':['sum','count'],'score':['count']})
tbl_failed_assessment_Total.columns = ['count_failed_assessment_Total']
tbl_failed_assessment_Total = tbl_failed_assessment_Total.reset_index()

# create distinct list
studentAssessment2_distinct=studentAssessment2[['id_student','code_module','code_presentation']]
studentAssessment2_distinct=studentAssessment2_distinct.drop_duplicates()

# merge
studentAssessment2_agg1 = studentAssessment2_distinct \
    .merge(tbl_total_weighted_score, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_total_score, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_total_assessment, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_passed_assessment_CMA, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_passed_assessment_TMA, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_passed_assessment_Exam, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_passed_assessment_Total, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_failed_assessment_CMA, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_failed_assessment_TMA, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_failed_assessment_Exam, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_failed_assessment_Total, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) 

# add rate_passed_assessment
studentAssessment2_agg1['rate_passed_assessment'] = studentAssessment2_agg1['count_passed_assessment_Total']*studentAssessment2_agg1['count_total_assessment']

# export to csv
# studentAssessment2_agg1.to_csv('DatasetForModel_StudentAssessmentInfo_week16.csv',index=True,header=True)

### Data Preparation: Student Assessment

##### What we want to prepare

##### Aggregated Dimension
* id_student
* code_module
* code_presentation

##### Aggregated Metrics
* sum_click_total
* sum_click_resource
* sum_click_oucontent
* sum_click_url
* sum_click_homepage
* sum_click_subpage
* sum_click_glossary
* sum_click_forumng
* sum_click_oucollaborate
* sum_click_dataplus
* sum_click_quiz
* sum_click_ouelluminate
* sum_click_sharedsubpage
* sum_click_questionnaire
* sum_click_page
* sum_click_externalquiz
* sum_click_ouwiki
* sum_click_dualpane
* sum_click_repeatactivity
* sum_click_folder
* sum_click_htmlactivity

In [16]:
# import data
studentVle = pd.read_csv('studentVle.csv')
studentVle2 = studentVle
vle = pd.read_csv('vle.csv')

# merge data
studentVle3 = studentVle2.merge(vle, how='left', \
                                                 left_on=["code_module", "code_presentation","id_site"], \
                                                 right_on=["code_module", "code_presentation","id_site"])
#studentVle3.to_csv('studentVle3.csv',index=True,header=True)

# add date_group/ weeks
studentVle3['_date_group'] = pd.Series([], dtype=float)
studentVle3['_date_group'] = studentVle3['date']
studentVle3.loc[studentVle3['date'] <0, '_date_group'] = studentVle3['_date_group'].apply(lambda x: math.ceil(float(x/7)))
studentVle3.loc[studentVle3['date'] >=0, '_date_group'] = studentVle3['_date_group'].apply(lambda x: math.floor(float((x/7)+1)))

# for EEE Course, Cut off filter at Week 16
studentVle3 = studentVle3[(studentVle3['_date_group']<=16)]
# studentVle3.to_csv('studentVle3_week16.csv',index=True,header=True)
# for EEE Course, Cut off filter at Week 16

In [17]:
# Feature Engineering
# transforming multi-rows input to individual vertical columns

tbl_sum_click_total_precoursestart=studentVle3\
            [(studentVle3['_date_group']<=0)].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_total_precoursestart.columns = ['sum_click_total_precoursestart']
tbl_sum_click_total_precoursestart = tbl_sum_click_total_precoursestart.reset_index()

tbl_sum_click_total=studentVle3.\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_total.columns = ['sum_click_total']
tbl_sum_click_total = tbl_sum_click_total.reset_index()

tbl_sum_click_resource=studentVle3\
            [(studentVle3['activity_type'].isin(['resource']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
            #agg({'_flag_assessment_passed':['sum','count'],'score':['count']})
tbl_sum_click_resource.columns = ['sum_click_resource']
tbl_sum_click_resource = tbl_sum_click_resource.reset_index()
#print(tbl_sum_click_resource)

tbl_sum_click_oucontent=studentVle3\
            [(studentVle3['activity_type'].isin(['oucontent']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_oucontent.columns = ['sum_click_oucontent']
tbl_sum_click_oucontent = tbl_sum_click_oucontent.reset_index()

tbl_sum_click_url=studentVle3\
            [(studentVle3['activity_type'].isin(['url']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_url.columns = ['sum_click_url']
tbl_sum_click_url = tbl_sum_click_url.reset_index()

tbl_sum_click_homepage=studentVle3\
            [(studentVle3['activity_type'].isin(['homepage']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_homepage.columns = ['sum_click_homepage']
tbl_sum_click_homepage = tbl_sum_click_homepage.reset_index()

tbl_sum_click_subpage=studentVle3\
            [(studentVle3['activity_type'].isin(['subpage']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_subpage.columns = ['sum_click_subpage']
tbl_sum_click_subpage = tbl_sum_click_subpage.reset_index()

tbl_sum_click_glossary=studentVle3\
            [(studentVle3['activity_type'].isin(['glossary']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_glossary.columns = ['sum_click_glossary']
tbl_sum_click_glossary = tbl_sum_click_glossary.reset_index()

tbl_sum_click_forumng=studentVle3\
            [(studentVle3['activity_type'].isin(['forumng']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_forumng.columns = ['sum_click_forumng']
tbl_sum_click_forumng = tbl_sum_click_forumng.reset_index()

tbl_sum_click_oucollaborate=studentVle3\
            [(studentVle3['activity_type'].isin(['oucollaborate']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_oucollaborate.columns = ['sum_click_oucollaborate']
tbl_sum_click_oucollaborate = tbl_sum_click_oucollaborate.reset_index()

tbl_sum_click_dataplus=studentVle3\
            [(studentVle3['activity_type'].isin(['dataplus']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_dataplus.columns = ['sum_click_dataplus']
tbl_sum_click_dataplus = tbl_sum_click_dataplus.reset_index()

tbl_sum_click_quiz=studentVle3\
            [(studentVle3['activity_type'].isin(['quiz']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_quiz.columns = ['sum_click_quiz']
tbl_sum_click_quiz = tbl_sum_click_quiz.reset_index()

tbl_sum_click_ouelluminate=studentVle3\
            [(studentVle3['activity_type'].isin(['ouelluminate']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_ouelluminate.columns = ['sum_click_ouelluminate']
tbl_sum_click_ouelluminate = tbl_sum_click_ouelluminate.reset_index()

tbl_sum_click_sharedsubpage=studentVle3\
            [(studentVle3['activity_type'].isin(['sharedsubpage']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_sharedsubpage.columns = ['sum_click_sharedsubpage']
tbl_sum_click_sharedsubpage = tbl_sum_click_sharedsubpage.reset_index()

tbl_sum_click_questionnaire=studentVle3\
            [(studentVle3['activity_type'].isin(['questionnaire']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_questionnaire.columns = ['sum_click_questionnaire']
tbl_sum_click_questionnaire = tbl_sum_click_questionnaire.reset_index()

tbl_sum_click_page=studentVle3\
            [(studentVle3['activity_type'].isin(['page']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_page.columns = ['sum_click_page']
tbl_sum_click_page = tbl_sum_click_page.reset_index()

tbl_sum_click_externalquiz=studentVle3\
            [(studentVle3['activity_type'].isin(['externalquiz']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_externalquiz.columns = ['sum_click_externalquiz']
tbl_sum_click_externalquiz = tbl_sum_click_externalquiz.reset_index()

tbl_sum_click_ouwiki=studentVle3\
            [(studentVle3['activity_type'].isin(['ouwiki']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_ouwiki.columns = ['sum_click_ouwiki']
tbl_sum_click_ouwiki = tbl_sum_click_ouwiki.reset_index()

tbl_sum_click_dualpane=studentVle3\
            [(studentVle3['activity_type'].isin(['dualpane']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_dualpane.columns = ['sum_click_dualpane']
tbl_sum_click_dualpane = tbl_sum_click_dualpane.reset_index()

tbl_sum_click_repeatactivity=studentVle3\
            [(studentVle3['activity_type'].isin(['repeatactivity']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_repeatactivity.columns = ['sum_click_repeatactivity']
tbl_sum_click_repeatactivity = tbl_sum_click_repeatactivity.reset_index()

tbl_sum_click_folder=studentVle3\
            [(studentVle3['activity_type'].isin(['folder']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_folder.columns = ['sum_click_folder']
tbl_sum_click_folder = tbl_sum_click_folder.reset_index()

tbl_sum_click_htmlactivity=studentVle3\
            [(studentVle3['activity_type'].isin(['htmlactivity']))].\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_htmlactivity.columns = ['sum_click_htmlactivity']
tbl_sum_click_htmlactivity = tbl_sum_click_htmlactivity.reset_index()


studentVle3_distinct=studentVle3[['id_student','code_module','code_presentation']]
studentVle3_distinct=studentVle3_distinct.drop_duplicates()


studentVle3_agg1 = studentVle3_distinct\
    .merge(tbl_sum_click_total, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_total_precoursestart, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_resource, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_oucontent, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_url, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_homepage, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_subpage, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_glossary, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_forumng, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_oucollaborate, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_dataplus, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_quiz, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_ouelluminate, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_sharedsubpage, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_questionnaire, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_page, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_externalquiz, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_ouwiki, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_dualpane, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_repeatactivity, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_folder, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) \
    .merge(tbl_sum_click_htmlactivity, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"])

# export to csv
# studentVle3_agg1.to_csv('DatasetForModel_StudentVleInfo_week16.csv',index=True,header=True)


In [18]:
# checks on the dates
# XXX=studentVle3[(studentVle3['code_module'].isin(['EEE']))]
# print(XXX['_date_group'].min()) #-2
# print(XXX['_date_group'].max()) #16

##### What we want to prepare

##### Tables Merging
* studentInfo3
* studentAssessment2_agg1
* studentVle3_agg1

##### Aggregated Dimension
* id_student
* code_module
* code_presentation

##### Aggregated Metrics
* all the rest of the metrics

In [19]:
DatasetForModel_TableOne = studentInfo3\
    .merge(studentAssessment2_agg1, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(studentVle3_agg1, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) 

##### Weighted Score Total Denominator Value
* AAA 100
* BBB 100
* CCC 200
* DDD 200
* EEE 100
* FFF 100
* GGG ZERO

In [20]:
# export to csv
#DatasetForModel_TableOne.to_csv('DatasetForModel_TableOne.csv',index=True,header=True)

#DatasetForModel_TableOne_EEE2014J = DatasetForModel_TableOne\
#[DatasetForModel_TableOne['code_module'].isin(['EEE'])&\
# DatasetForModel_TableOne['code_presentation'].isin(['2014J'])]
#DatasetForModel_TableOne_EEE2014J

#DatasetForModel_TableOne_EEE2014J['final_weighted_score'] = DatasetForModel_TableOne_EEE2014J['sum_weighted_score']/100

# export to csv
# DatasetForModel_TableOne_EEE2014J.to_csv('DatasetForModel_TableOne_EEE2014J_Week16.csv',index=True,header=True)



# rename. lazy to type
#EEE2014J=DatasetForModel_TableOne_EEE2014J
#EEE2014J_desc=EEE2014J.describe()

# for checkings only 
# EEE2014J_desc.to_csv('EEE2014J_desc.csv',index=True,header=True)
# EEE2014J_desc

# EEE2014J.info()
# 50 columns

#final_result
#_flag_actual_dropout
#Withdrawn

#EEE2014Jx = EEE2014J[(EEE2014J['final_result'].isin(['Withdrawn'])) & (EEE2014J['_flag_actual_dropout'].isin([0]))]
#71 
#EEE2014J = EEE2014J[~((EEE2014J['final_result'].isin(['Withdrawn'])) & (EEE2014J['_flag_actual_dropout'].isin([0])))]
#1117

# exclude withdrawal pre semester start

#EEE2014J['_flag_final_result'] = pd.Series([], dtype=int)
#EEE2014J.loc[(EEE2014J['final_result'].isin(['Withdrawn']))|(EEE2014J['final_result'].isin(['Fail'])), '_flag_final_result'] = 1
#EEE2014J.loc[(EEE2014J['final_result'].isin(['Distinction']))|(EEE2014J['final_result'].isin(['Pass'])), '_flag_final_result'] = 0

# checks
# EEE2014J.head()
# EEE2014J.to_csv('EEE2014J_Week16.csv',index=True,header=True)

In [21]:
##############
# all EEEs

DatasetForModel_TableOne_EEE = DatasetForModel_TableOne\
[DatasetForModel_TableOne['code_module'].isin(['EEE'])]

DatasetForModel_TableOne_EEE['final_weighted_score'] = DatasetForModel_TableOne_EEE['sum_weighted_score']/100

EEE=DatasetForModel_TableOne_EEE
EEE_desc=EEE.describe()
# EEE_desc.to_csv('EEE_desc.csv',index=True,header=True)

EEE = EEE[~((EEE['final_result'].isin(['Withdrawn'])) & (EEE['_flag_actual_dropout'].isin([0])))]

EEE['_flag_final_result'] = pd.Series([], dtype=int)
EEE.loc[(EEE['final_result'].isin(['Withdrawn']))|(EEE['final_result'].isin(['Fail'])), '_flag_final_result'] = 0
EEE.loc[(EEE['final_result'].isin(['Distinction']))|(EEE['final_result'].isin(['Pass'])), '_flag_final_result'] = 1

EEE.to_csv('EEE_Week16.csv',index=True,header=True)

<ipython-input-21-157677cefdfe>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DatasetForModel_TableOne_EEE['final_weighted_score'] = DatasetForModel_TableOne_EEE['sum_weighted_score']/100
<ipython-input-21-157677cefdfe>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EEE['_flag_final_result'] = pd.Series([], dtype=int)
C:\Users\EeLian\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i